# Serve R with Seldon-Core and Kubernetes

**Install s2i to ~/tmp**

In [ ]:
%%bash
mkdir tmp
cd tmp
wget https://github.com/openshift/source-to-image/releases/download/v1.1.13/source-to-image-v1.1.13-b54d75d3-linux-amd64.tar.gz
tar -xvf source-to-image-v1.1.13-b54d75d3-linux-amd64.tar.gz

**Create environment file for s2i**

In [ ]:
!mkdir ./serving/.s2i

In [ ]:
%%writefile ./serving/.s2i/environment
MODEL_NAME=babyweight.R
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0

**Create install.R file for dependencies**

In [ ]:
%%writefile ./serving/install.R
install.packages("glmnet")
install.packages("caret")

**Copy Model File to Deploy for Cloud Storage**

In [ ]:
%%bash
export STORAGE_BUCKET=r-demo-data
export MODEL_VERSION=20190408_121335

gsutil cp gs://$STORAGE_BUCKET/$MODEL_VERSION/model.rds ./serving
gsutil cp gs://$STORAGE_BUCKET/$MODEL_VERSION/preproc.rds ./serving

**Create R Runtime Model File**

In [ ]:
%%writefile ./serving/babyweight.R
library(methods)
library(glmnet)

predict.babyweight <- function(babyweight,newdata=list()) {
    # center and scale data
    newdata[,1:5] <- sapply(newdata[,1:5], as.numeric)
    newdata <- predict(babyweight$preproc, newdata)

    # set factor data types
    newdata[, c("is_male", "child_race")] <- sapply(newdata[, c("is_male", "child_race")] , as.factor)

    # inverse of log transform
    exp(predict(babyweight$model, newdata))
}

new_babyweight <- function(modelfile, preprocfile) {
    model <- readRDS(modelfile)
    preproc <- readRDS(preprocfile)
    structure(list(model=model,preproc=preproc), class = "babyweight")
}

initialise_seldon <- function(params) {
    new_babyweight("model.rds", "preproc.rds")
}

**Build R Image**

In [ ]:
%%bash
export PATH=$PATH:$PWD/tmp
export PROJECT_ID=$(gcloud config get-value project)
cd ./serving
s2i build . seldonio/seldon-core-s2i-r:0.1 gcr.io/$PROJECT_ID/r-babyweight:latest

**Test Serving Container Locally**

In [ ]:
%%bash
export PROJECT_ID=$(gcloud config get-value project)
docker run --name "babyweight_predictor" -d --rm -p 5000:5000 gcr.io/$PROJECT_ID/r-babyweight:latest

In [ ]:
%%bash
curl -g http://localhost:5000/predict --data-urlencode \
        'json={"data": 
                {"names": 
                    ["is_male", 
                     "child_race",  
                     "mother_age", 
                     "father_age", 
                     "gestation_weeks"],
                 "ndarray": 
                     [[1,1,40,44,39]]}}'

In [ ]:
# view logs for debugging
!docker logs babyweight_predictor

In [ ]:
# stop container
!docker rm babyweight_predictor --force

In [ ]:
!gcloud auth configure-docker

In [ ]:
%%bash
export PROJECT_ID=$(gcloud config get-value project)
docker push gcr.io/$PROJECT_ID/r-babyweight:latest

**Create a 2 node kubernetes cluster on Google Kubernetes Engine**

In [ ]:
!gcloud container clusters create babyweight-cluster --num-nodes=2 --region us-central1

In [ ]:
# Verify Deployment
!gcloud compute instances list

**Deploy serving application to GKE**

In [ ]:
%%bash
export PROJECT_ID=$(gcloud config get-value project)
kubectl run babyweight --image=gcr.io/$PROJECT_ID/r-babyweight:latest --port 5000

In [ ]:
# verify deployment
!kubectl get pods

**Expose the application on port 80, get the external IP, and send a request**

In [ ]:
!kubectl expose deployment babyweight --type=LoadBalancer --port 80 --target-port 5000

In [ ]:
!kubectl get service

In [ ]:
%%bash
export EXTERNAL_IP=35.238.0.217
curl -g http://$EXTERNAL_IP:80/predict --data-urlencode \
        'json={"data": 
                {"names": 
                    ["is_male", 
                     "child_race",  
                     "mother_age", 
                     "father_age", 
                     "gestation_weeks"],
                 "ndarray": 
                     [[1,1,40,44,39]]}}'